# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 9 2022 8:30AM,243716,10,0085925778,ISDIN Corporation,Released
1,Aug 9 2022 8:30AM,243716,10,0085928352,ISDIN Corporation,Released
2,Aug 9 2022 8:30AM,243716,10,0085928425,ISDIN Corporation,Released
3,Aug 9 2022 8:30AM,243716,10,0085928434,ISDIN Corporation,Released
4,Aug 9 2022 8:30AM,243716,10,0085928451,ISDIN Corporation,Released
5,Aug 9 2022 8:30AM,243716,10,0085928459,ISDIN Corporation,Released
6,Aug 9 2022 8:30AM,243716,10,0085928467,ISDIN Corporation,Released
7,Aug 9 2022 8:30AM,243716,10,0085928471,ISDIN Corporation,Released
8,Aug 9 2022 8:30AM,243716,10,0085928480,ISDIN Corporation,Released
9,Aug 9 2022 8:30AM,243716,10,0085928498,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,243714,Released,1
21,243715,Released,1
22,243716,Released,15
23,243718,Released,18
24,243719,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243714,NaN,1.0
243715,NaN,1.0
243716,NaN,15.0
243718,NaN,18.0
243719,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243612,1.0,0.0
243617,1.0,0.0
243627,0.0,1.0
243662,1.0,0.0
243677,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243612,1,0
243617,1,0
243627,0,1
243662,1,0
243677,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243612,1,0
1,243617,1,0
2,243627,0,1
3,243662,1,0
4,243677,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243612,1,
1,243617,1,
2,243627,,1
3,243662,1,
4,243677,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation
15,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation
33,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation
39,Aug 8 2022 4:27PM,243719,102,"Senseonics, Incorporated"
40,Aug 8 2022 4:21PM,243715,102,"Senseonics, Incorporated"
41,Aug 8 2022 4:13PM,243714,102,"Senseonics, Incorporated"
42,Aug 8 2022 4:06PM,243713,102,"Senseonics, Incorporated"
43,Aug 8 2022 4:00PM,243711,19,Eli Lilly and Company
44,Aug 8 2022 3:43PM,243708,20,"ACI Healthcare USA, Inc."
59,Aug 8 2022 3:28PM,243707,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,,15
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,,18
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,,6
3,Aug 8 2022 4:27PM,243719,102,"Senseonics, Incorporated",,1
4,Aug 8 2022 4:21PM,243715,102,"Senseonics, Incorporated",,1
5,Aug 8 2022 4:13PM,243714,102,"Senseonics, Incorporated",,1
6,Aug 8 2022 4:06PM,243713,102,"Senseonics, Incorporated",,1
7,Aug 8 2022 4:00PM,243711,19,Eli Lilly and Company,,1
8,Aug 8 2022 3:43PM,243708,20,"ACI Healthcare USA, Inc.",,15
9,Aug 8 2022 3:28PM,243707,10,"Nextsource Biotechnology, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15,
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18,
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6,
3,Aug 8 2022 4:27PM,243719,102,"Senseonics, Incorporated",1,
4,Aug 8 2022 4:21PM,243715,102,"Senseonics, Incorporated",1,
5,Aug 8 2022 4:13PM,243714,102,"Senseonics, Incorporated",1,
6,Aug 8 2022 4:06PM,243713,102,"Senseonics, Incorporated",1,
7,Aug 8 2022 4:00PM,243711,19,Eli Lilly and Company,1,
8,Aug 8 2022 3:43PM,243708,20,"ACI Healthcare USA, Inc.",15,
9,Aug 8 2022 3:28PM,243707,10,"Nextsource Biotechnology, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15,
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18,
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6,
3,Aug 8 2022 4:27PM,243719,102,"Senseonics, Incorporated",1,
4,Aug 8 2022 4:21PM,243715,102,"Senseonics, Incorporated",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15.0,NaN
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18.0,NaN
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6.0,NaN
3,Aug 8 2022 4:27PM,243719,102,"Senseonics, Incorporated",1.0,NaN
4,Aug 8 2022 4:21PM,243715,102,"Senseonics, Incorporated",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 9 2022 8:30AM,243716,10,ISDIN Corporation,15.0,0.0
1,Aug 9 2022 8:30AM,243718,10,ISDIN Corporation,18.0,0.0
2,Aug 9 2022 8:30AM,243712,10,ISDIN Corporation,6.0,0.0
3,Aug 8 2022 4:27PM,243719,102,"Senseonics, Incorporated",1.0,0.0
4,Aug 8 2022 4:21PM,243715,102,"Senseonics, Incorporated",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1705914,43.0,0.0
102,974861,4.0,0.0
16,731024,1.0,2.0
19,731018,2.0,1.0
20,1218467,79.0,3.0
21,243662,0.0,1.0
22,243627,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1705914,43.0,0.0
1,102,974861,4.0,0.0
2,16,731024,1.0,2.0
3,19,731018,2.0,1.0
4,20,1218467,79.0,3.0
5,21,243662,0.0,1.0
6,22,243627,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,43.0,0.0
1,102,4.0,0.0
2,16,1.0,2.0
3,19,2.0,1.0
4,20,79.0,3.0
5,21,0.0,1.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,43.0
1,102,Released,4.0
2,16,Released,1.0
3,19,Released,2.0
4,20,Released,79.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,19,20,21,22
Status,,,,,,,
Executing,0.0,0.0,2.0,1.0,3.0,1.0,0.0
Released,43.0,4.0,1.0,2.0,79.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,19,20,21,22
0,Executing,0.0,0.0,2.0,1.0,3.0,1.0,0.0
1,Released,43.0,4.0,1.0,2.0,79.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,19,20,21,22
0,Executing,0.0,0.0,2.0,1.0,3.0,1.0,0.0
1,Released,43.0,4.0,1.0,2.0,79.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()